In [84]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import plotly.express as px


In [85]:
promo = pd.read_excel("../data/promo_history.xlsx", index_col=0)
sales = pd.read_csv("../data/sales_history.csv", index_col=0)
promo = promo.drop(columns=0)
sales['sale_dt'] = pd.to_datetime(sales['sale_dt'])


/home/royalcat/projects/hacks/accenture-promo-hack/venv/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [86]:
sales['skutertiaryid'].unique()

array([7182, 7189, 7193, 7194, 7205, 7232, 7234, 7236, 7247, 7238])

In [87]:
items = sales['skutertiaryid'].unique()
prices = {}
for itemid in items:
    item_sales = sales[sales['skutertiaryid'] == itemid]
    prices[itemid] = (item_sales['salerevenuerub']/item_sales['soldpieces']).replace([np.inf, -np.inf], np.nan).max()

In [88]:
sales_sum = sales.drop(['posid'], axis=1).groupby(['skutertiaryid', 'sale_dt']).sum().reset_index()

In [90]:
sales_sum['week'] = sales_sum['sale_dt'].dt.isocalendar().week
sales_sum['year'] = sales_sum['sale_dt'].dt.isocalendar().year
sales_sum = sales_sum.drop('sale_dt', axis=1)

In [91]:
sales_sum.to_csv('../data/sales_sum_history.csv')

In [92]:
promo_sum = promo.copy(deep=True)
promo_sum['start_year'] = promo_sum['start_dttm'].dt.isocalendar().year
promo_sum['start_week'] = promo_sum['start_dttm'].dt.isocalendar().week
promo_sum['end_year'] = promo_sum['end_dttm'].dt.isocalendar().year
promo_sum['end_week'] = promo_sum['end_dttm'].dt.isocalendar().week
promo_sum = promo_sum.drop(['start_dttm', 'end_dttm', 'promotypeid'], axis=1).sort_values('skutertiaryid')

In [94]:
promo_revenue = []
promo_soldpieces = []

for i, row in tqdm(promo_sum.iterrows()):
    promo_sales = sales_sum[(sales_sum['week'] >= row['start_week']) & (sales_sum['week'] <= row['end_week']) & \
                            ((sales_sum['year'] == row['start_year']) | (sales_sum['year'] == row['end_year']))]
    promo_revenue.append(prices[row['skutertiaryid']]  * row['chaindiscountvalue'] * (promo_sales['soldpieces'].sum()))
    promo_soldpieces.append(promo_sales['soldpieces'].sum())

promo_sum['revenue'] = promo_revenue
promo_sum['soldpieces'] = promo_soldpieces


103it [00:00, 566.86it/s]


In [95]:
promo_sum = promo_sum[promo_sum['soldpieces'] != 0]

In [96]:
promo_sum_add = []
for i, row in promo_sum.iterrows():
    promo_time = row['end_week'] - row['start_week']
    if promo_time > 1:
        row['revenue'] = sales_sum[(sales_sum['skutertiaryid'] == row['skutertiaryid']) &\
                                   (sales_sum['week'] == row['start_week'])]['salerevenuerub'].sum()
        new_row = row.copy(deep=True)
        new_row['start_week'] = new_row['start_week'] + 1
        new_row['revenue'] = sales_sum[(sales_sum['skutertiaryid'] == new_row['skutertiaryid']) &\
                            (sales_sum['week'] == new_row['start_week'])]['salerevenuerub'].sum()
        promo_sum_add.append(new_row)

In [97]:
promo_sum = promo_sum.append(promo_sum_add).drop(['end_week', 'end_year'], axis=1)

In [102]:
promo_sum.to_csv('../data/promo_sum_history.csv')

In [112]:
sales_sum.sample(3)

,skutertiaryid,salerevenuerub,soldpieces,week,year
1370,7238,2.331297e+06,105391,28,2019
1392,7238,1.866580e+06,64965,46,2019
824,7232,5.706374e+06,118321,24,2018


In [111]:
promo_sum.sample(3)

,skutertiaryid,chaindiscountvalue,start_year,start_week,revenue,soldpieces,sold_added
49,7189,0.134429,2019,45,1.446190e+07,2394781,1.652975e+06
16,7205,0.165380,2019,19,3.195915e+07,4025235,1.378326e+06
38,7236,0.186274,2019,37,4.990503e+06,2679623,2.352737e+06
52,7182,0.206007,2019,46,6.976407e+07,2301296,-1.530981e+06
21,7193,0.300475,2019,21,2.992706e+08,11786149,8.682593e+06


In [108]:
promo_sum[promo_sum['skutertiaryid'] == 7182]

,skutertiaryid,chaindiscountvalue,start_year,start_week,revenue,soldpieces,sold_added
52,7182,0.206007,2019,46,6.976407e+07,2301296,-1.530981e+06
43,7182,0.206215,2019,42,7.852176e+07,2587576,-1.244701e+06
39,7182,0.205024,2019,38,6.393776e+07,2119216,-1.713061e+06
27,7182,0.146242,2019,26,3.846121e+07,1787199,-2.045078e+06
14,7182,0.198616,2019,17,1.681596e+08,5753487,1.921210e+06
28,7182,0.195177,2019,29,1.044350e+08,3636140,-1.961366e+05
3,7182,0.233531,2019,7,1.079884e+08,3142354,-6.899226e+05
7,7182,0.230109,2019,11,1.062178e+08,3136796,-6.954806e+05
14,7182,0.198616,2019,18,1.431900e+07,5753487,1.921210e+06
3,7182,0.233531,2019,8,1.042321e+07,3142354,-6.899226e+05


In [46]:
promo_sum.head(5)

,skutertiaryid,chaindiscountvalue,start_year,start_week,revenue,soldpieces
52,7182,0.206007,2019,46,6.976407e+07,2301296
43,7182,0.206215,2019,42,7.852176e+07,2587576
39,7182,0.205024,2019,38,6.393776e+07,2119216
27,7182,0.146242,2019,26,3.846121e+07,1787199
14,7182,0.198616,2019,17,1.681596e+08,5753487


In [56]:
sales_sum.head(5)

,skutertiaryid,sale_dt,salerevenuerub,soldpieces,week,year
0,7182,2018-04-23,4.067299e+06,48862,17,2018
1,7182,2018-04-30,7.811600e+05,9381,18,2018
2,7182,2018-05-01,3.837060e+06,46044,18,2018
3,7182,2018-05-07,4.894995e+06,63986,19,2018
4,7182,2018-05-14,4.523049e+06,57705,20,2018


In [123]:
normal_sold = {}
for itemid in sales_sum['skutertiaryid'].unique():
    normal_sold[itemid] = sales_sum[sales_sum['skutertiaryid'] == itemid]['salerevenuerub'].median()


In [124]:
normal_sold

{7182: 3832276.599599929,
 7189: 741805.5060000226,
 7193: 3103555.506870055,
 7194: 8173341.8672401095,
 7205: 2646909.171431901,
 7232: 5340534.957000011,
 7234: 13371277.923671968,
 7236: 326886.3552000041,
 7238: 1268322.3659998327,
 7247: 308337.8753448069}

In [125]:
sold_added = []
for i, row in promo_sum.iterrows():
    sold_added.append((row['soldpieces']/normal_sold[row['skutertiaryid']]))
promo_sum['sold_added'] = sold_added

In [126]:
promo_sum.sort_values('sold_added')

,skutertiaryid,chaindiscountvalue,start_year,start_week,revenue,soldpieces,sold_added
58,7234,0.171371,2019,49,4.536458e+07,2233920,0.167069
54,7234,0.175490,2019,46,4.785586e+07,2301296,0.172107
1,7234,0.239229,2019,4,3.136620e+07,2543043,0.190187
1,7234,0.239229,2019,3,7.209066e+07,2543043,0.190187
45,7234,0.170127,2019,42,5.216486e+07,2587576,0.193517
...,...,...,...,...,...,...,...
15,7236,0.185099,2019,19,4.239683e+06,2290926,7.008326
18,7189,0.173002,2019,20,3.828410e+06,5784190,7.797448
18,7189,0.173002,2019,19,4.495296e+07,5784190,7.797448
38,7236,0.186274,2019,37,4.990503e+06,2679623,8.197415


In [127]:
promo_sum.to_csv('../data/promo_sum_history.csv')